In [1]:
import pandas as pd  
from unidecode import unidecode
import re

In [16]:
# Function to clean text from PDFs and add some delimiters for further processing

def limpiador1(text: str) -> str:
    
    text = re.sub(r'\n\d{1,3}', '', text) # Remove line breaks followed by one to three digits
    text = re.sub(r'\.\s\n\n', '. @@@ ', text) # Replace periods followed by two line breaks with '. @@@ '
    text = re.sub(r'\n\.', '', text) # Remove line breaks followed by a period
    text = re.sub(r'\s\n\s', ' ', text) # Replace spaces surrounding line breaks with a single space
    text = re.sub(r'\n{1,100}', ' ', text) # Replace multiple line breaks with a single space
    text = re.sub(r'SESION PLENARIA ORDINARIA [0-9]{1,3}', '  ', text) # Replace 'SESION PLENARIA ORDINARIA' followed by one to three digits with two spaces
    text = re.sub(r'\d{1,3} SESION PLENARIA ORDINARIA', '  ', text) # Replace one to three digits followed by 'SESION PLENARIA ORDINARIA' with two spaces
    text = re.sub(r'\s{2,100}', ' ', text) # Replace multiple spaces with a single space
    text = re.sub(r'Intervino', '___Intervino', text) # Add '___Intervino' after the occurrence of 'Intervino'
    
    # Return the processed text
    return text


In [3]:
df = pd.read_csv('actas_concejo.csv')

print(limpiador1(df['texto'][0][:70]))
limpiador1(df['texto'][0][:70])

CONCEJO DE MEDELLIN SESION ORDINARIA ACTA 670 Junio 9 de 2015 


'CONCEJO DE MEDELLIN SESION ORDINARIA ACTA 670 Junio 9 de 2015 '

In [17]:
# Patterns for the header of the minutes.

pattern_acta: str = r'(\bACTA\s?[\d\w]{1,4})'
pattern_tipo_sesion: str = r'(\b(SESION|sEslou|sEsron)\s[\s\S]+?)\sACTA'
pattern_fecha: str = r'(\b[A-Z][a-z]+?[\s\S]+?\d{4})' 

# Content patterns

pattern_intervino_texto: str = r'(___Intervino[\S\s]+?:[\S\s]+?)___'
pattern_intervino: str = r'___Intervino[\S\s]+?([A-Z][a-z]*?[\S\s]+?):'
pattern_intervino2: str = r'___Intervino\s?([\S\s]+?):'
pattern_intervencion: str = r'___Intervino.+?:'



In [ ]:
# Debugging of the dataframe.

df_depurado = pd.DataFrame()
nueva_lista_actas: list = []
numero_acta: list = []

for i, acta in enumerate(df['texto']):
    
    if type(acta) == str and len(acta) > 5000:
    
        acta = limpiador1(acta)
    
        acta_ = re.search(pattern_acta, acta).group(1)
        
        if acta_ not in numero_acta:
            nueva_lista_actas.append(acta)
        
            print(f'''
                index: {i}
                acta: {acta_}
                _______________________________________________________________
                ''')
            print(len(nueva_lista_actas))

print(f'Numero de actas ANTES de la depuracion: {len(df["texto"])}')
df_depurado['texto'] = nueva_lista_actas
print(f'Numero de actas DESPUES de la depuracion: {len(df_depurado["texto"])}')

In [ ]:
# "Verification of header data in all minutes."

for i, acta in enumerate(df_depurado['texto']):
    print(i)

    acta = acta[:5000]
    
    acta_ = re.search(pattern_acta, acta).group(1)
    tipo_sesion = re.search(pattern_tipo_sesion, acta).group(1)
    fecha = re.search(pattern_fecha, acta).group(1)
    
    print(f'''
        - Index: {i}
        - Tipo de dato: {type(acta)}
        - Longitud del texto: {len(acta)}
        - Texto del encabezado: {acta[:70]}
        - Acta: {acta_}
        - Tipo de sesion: {tipo_sesion}
        - Fecha: {fecha}
        ____________________________________________________________________________

          ''')
    

In [7]:
intervencion_intervienen: list = re.findall(pattern_intervino_texto, df_depurado['texto'][0])
len(intervencion_intervienen)

13

In [ ]:
# We test if everything is working correctly.
intervenciones_acta = re.findall(pattern_intervino_texto, df_depurado['texto'][61])

intervencion = ''
intervino = ''

print(type(intervenciones_acta))
print(len(intervenciones_acta))

for i, inter in enumerate(intervenciones_acta):
    print(inter)
    intervencion = re.sub(pattern_intervencion, '', inter)
    
    if re.search(pattern_intervino, inter):
        intervino =  re.search(pattern_intervino, inter).group(1)
    else:
        intervino =  re.search(pattern_intervino2, inter).group(1)

    print(f'''
          - Index:  {i}
          - Intervencion: {intervencion[:100]}
          - intervino: {intervino}
          ________________________________________________________________
          ''')

In [ ]:
# We build the dataframe with the interventions.

dict_intervenciones: dict = {
    'acta': [],
    'fecha': [],
    'sesion': [],
    'intervino': [],
    'intervencion': []
}

intervencion_intervienen: list = [] 
acta_numero: list = []
acta_fecha: list = []
sesion: list = []

intervino_list: list = []
intervencion_list: list = []

acta_n: str = ''
tipo_sesion: str = ''
fecha: str = ''
texto_intervencion: str = ''
texto_interviene: str = ''

for i, acta in enumerate(df_depurado['texto']):

    acta_n = re.search(pattern_acta, acta).group(1).title()
    tipo_sesion = re.search(pattern_tipo_sesion, acta).group(1).title()
    fecha = re.search(pattern_fecha, acta).group(1)
    
    intervencion_intervienen = re.findall(pattern_intervino_texto, acta)
    
    print(f'''
    - Index: {i}
    - Acta: {acta_n}
    - Tipo de sesion: {tipo_sesion}
    - Fecha: {fecha}
    - Intervenciones: {len(intervencion_intervienen)}
    ____________________________________________________________________________
    ''')
    
    for intervencion in intervencion_intervienen:
        acta_numero.append(acta_n)
        acta_fecha.append(fecha)
        sesion.append(tipo_sesion)

        texto_intervencion = re.sub(pattern_intervencion, '', intervencion)
        
        if re.search(pattern_intervino, intervencion):
            texto_interviene = re.search(pattern_intervino, intervencion).group(1)
        else:
            texto_interviene = re.search(pattern_intervino2, intervencion).group(1)
        
        intervencion_list.append(texto_intervencion)
        intervino_list.append(texto_interviene)
        
dict_intervenciones: dict = {
    'acta': acta_numero,
    'fecha': acta_fecha,
    'sesion': sesion,
    'intervino': intervino_list,
    'intervencion': intervencion_list
}

In [10]:
print(len(intervencion_list))
print(len(intervino_list))

19797
19797


In [11]:
df_intervenciones = pd.DataFrame(dict_intervenciones)
df_intervenciones.to_csv('intervenciones_concejo_medellin.csv',)

df_intervenciones.head(5)

,acta,fecha,sesion,intervino,intervencion
0,Acta 670,Junio 9 de 2015,Sesion Ordinaria,Oscar Guillermo Hoyos Giraldo,"""Para solicitar se someta a consideracion el ..."
1,Acta 670,Junio 9 de 2015,Sesion Ordinaria,Bernardo Alejandro Guerra Hoyos,"""Los problemas de Villa Cafe; el hecho que el..."
2,Acta 670,Junio 9 de 2015,Sesion Ordinaria,Maria Mercedes Mateos Larraona,"""Estuvimos en el estadio con alegria, porque ..."
3,Acta 670,Junio 9 de 2015,Sesion Ordinaria,Fabio Humberto Rivera Rivera,"""Lo primero es que me cogen a ""boca de jarra""..."
4,Acta 670,Junio 9 de 2015,Sesion Ordinaria,Luis Bernardo Velez Montoya,"""Me uno a la protesta que hace el concejal Os..."


In [18]:
# Function to search for keyword matches in a DataFrame

def buscador_and(palabras_claves, df):
    
    # Convert keywords to lowercase and remove accents
    palabras_claves = [unidecode(palabra).lower() for palabra in palabras_claves]
    
    # Function to check for keyword matches in a given text
    def verificador_de_palabras(texto):
        for palabra in palabras_claves:
            if palabra.lower() not in unidecode(texto).lower():
                return False
        return True  # Added for cases where there are no discrepancies
    
    # Create a 'coincidencia' column indicating keyword matches
    df['coincidencia'] = df['intervencion'].apply(verificador_de_palabras)
    
    # Filter the DataFrame based on the 'coincidencia' column
    condiciones = df['coincidencia'] == True
    df_filtrado = df[condiciones].copy()
    
    # Drop the 'coincidencia' column
    df_filtrado.drop('coincidencia', axis=1, inplace=True)
    
    return df_filtrado


In [19]:
# We choose a logical operator and keywords.

operador:str =  'AND'
palabras_claves: list = ['comuna', 'violencia']
expresion_regular: str = '|'.join(palabras_claves)

In [20]:
# Dataframe with the keywords and OR operator.
df_intervenciones_filtrado_or = df_intervenciones[df_intervenciones['intervencion'].str.contains(expresion_regular, case=False, regex=True)]
print(len(df_intervenciones_filtrado_or))

# Dataframe with the keywords and AND operator.
df_intervenciones_filtrado_and = buscador_and(palabras_claves, df_intervenciones)
print(len(df_intervenciones_filtrado_and))

df_intervenciones_filtrado_and.head(5)

4677
566


,acta,fecha,sesion,intervino,intervencion
58,Acta 090,Junio 13 de 2020,Sesion Plenaria Ordinaria,Luis Carlos Hernandez Castro,"""Quiero tocar tres puntos que me parecen impo..."
83,Acta 480,Julio 27 de 2022,Sesion Plenaria Ordinaria,"Seguridad y Convivencia, Jose Gerardo Acevedo ...","""La informacion que va a brindar la Secretari..."
85,Acta 480,Julio 27 de 2022,Sesion Plenaria Ordinaria,Babinton Dario Florez Moreno,"""Quiero de nuevo felicitar al concejal Luis C..."
88,Acta 480,Julio 27 de 2022,Sesion Plenaria Ordinaria,Daniel Duque Velasquez,"""La situacion de las familias de Medellin es ..."
89,Acta 480,Julio 27 de 2022,Sesion Plenaria Ordinaria,Nataly Velez Lopera,"""Un debate bien importante y bien traido el d..."


In [15]:
dict_intervenciones_filtradas: dict = {
    'OR': df_intervenciones_filtrado_or,
    'AND': df_intervenciones_filtrado_and
}

len(dict_intervenciones_filtradas['AND'])

566